In [1]:
import sys
from src.model import BoundaryDetector
from src.dataset import MovieDataset
import os
import math
import torch
from torch.utils.data import Dataset, DataLoader,random_split
import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [2]:
window_size=10

In [3]:
train_path = '/home/jolteon/eluvio_challenge/data/train'

In [4]:
dataset = MovieDataset(train_path,window_size=window_size)
dataset_length = len(dataset)
train_length = math.ceil(.8 *dataset_length)
val_length = dataset_length-train_length
train_set, val_set = random_split(dataset, [train_length, val_length], )#generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_set, batch_size = 4)
val_loader =  DataLoader(val_set, batch_size = 4)

In [5]:
print(dataset_length)
print(train_length)
print(val_length)

78331
62665
15666


In [6]:
#next(iter(train_loader))

In [7]:
model = BoundaryDetector(window_size=window_size)


In [8]:
trainer = pl.Trainer(gpus=1,
                     max_epochs=1,
                     val_check_interval=.5,
                     terminate_on_nan=True)
trainer.fit(model, train_dataloader = train_loader, val_dataloaders = val_loader)

MisconfigurationException: 
                You requested GPUs: [0]
                But your machine only has: []
            